<a href="https://colab.research.google.com/github/unicamp-dl/IA025_2022S1/blob/main/ex07/luiz_gontijo/Aula_7_Exerci%CC%81cio_Template_Luiz_Fernando_Gontijo_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
nome = 'Luiz Fernando da Costa Gontijo'
print(f'Meu nome é {nome}')

Meu nome é Luiz Fernando da Costa Gontijo


# Sobre a execução desse trabalho

Foram considerados alguns testes com uma fração da base de dados:

* 1 teste: 3 camadas e 2 relu -> PPL_val=1701.97
* 2 teste: 3 camadas e 2 tanh -> PPL_val=1540.55
* 3 teste: 3 camadas, 2 tanh e lr=1e-5 -> PPL_val=2020.16
* 4 teste: 3 camadas, 2 tanh, lr=3e-5, 2*neuronios nas duas camadas ocultas -> PPL_val=1782.67

Modelo final: 3 camadas, tanh, lr=3e-5 e 2*neuronios nas duas camadas ocultas 

O notebook no [link](https://drive.google.com/file/d/17OG3cti-cJcAry1tP3uw1Rf2ZEdFkY8p/view?usp=sharing) foi executado com 100.000.000 exemplos. O valor da perplexidade para os dados de teste encontrado foi igual a 188,547. O valor de perplexidade obtido neste notebook reflete somente 50.000.000 exemplos, não sendo o melhor resultado obtido no meu trabalho. 

Optei por essa estratégia com problemas no tempo de uso da GPU do colab. Inicialmente tentei executar com 400.000.000 exemplos, mas o tempo excedeu o limite. No entanto, pude notar que o valor da perplexidade tende a cair muito com o aumento dos dados de treino. 

#  Exercício: Modelo de Linguagem (Bengio 2003) - MLP + Embeddings

Neste exercício iremos treinar uma rede neural simples para prever a proxima palavra de um texto, data as palavras anteriores como entrada. Esta tarefa é chamada de "Modelagem da Língua".

Este dataset já possui um tamanho razoável e é bem provável que você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [2]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

     |████████████████████████████████| 4.2 MB 5.3 MB/s 
     |████████████████████████████████| 84 kB 4.2 MB/s 
     |████████████████████████████████| 596 kB 61.2 MB/s 
     |████████████████████████████████| 6.6 MB 56.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Importação dos pacotes

In [3]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook


In [4]:
# Check which GPU we are using
!nvidia-smi

Wed May 18 22:13:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

In [6]:
from typing import List


def tokenize(text: str, tokenizer):
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


# implementar uma janela deslizante, em que cada passo dessa janela serve como exemplos de treino
# exemplo:
# texto: eu gosto de comer pizza
# n = 2
# x = [eu, gosto] y =[de]
# x = [gosto, de] y =[comer]

class MyDataset():
    def __init__(self, texts: List[str], tokenizer, context_size: int):
        # Escreva seu código aqui
        self.texts = texts

        self.context_size = context_size

        self.target = []
        self.input = []
        for text in self.texts:
          tokens_key = tokenize(text, tokenizer)
          for i in range(len(tokens_key)-self.context_size):
            self.input.append(tokens_key[i:i+self.context_size])      
            self.target.append(tokens_key[i+self.context_size])

        self.input = torch.Tensor(self.input).long()
        self.target = torch.Tensor(self.target).long()
        

    def __len__(self):
        # Escreva seu código aqui
        dim = len(self.target)
        return dim

    def __getitem__(self, idx):
        # Escreva seu código aqui
        input = self.input[idx]
        target = self.target[idx]

        return input, target

## Teste se sua implementação do MyDataset está correta

In [7]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, context_size=3)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 5
print('passou no assert de tamanho do dataset')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[ 3396, 10303,   125],
     [ 1660,  5971,   785],
     [ 5971,   785,   125],
     [  785,   125,  1847],
     [  125,  1847, 13779]])

correct_first_batch_target = torch.LongTensor([13239,   125,  1847, 13779, 15616])

assert torch.equal(first_batch_input, correct_first_batch_input)
print('Passou no assert de input')
assert torch.equal(first_batch_target, correct_first_batch_target)
print('Passou no assert de target')

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

passou no assert de tamanho do dataset
Passou no assert de input
Passou no assert de target


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [8]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula7/sample_brwac.txt

--2022-05-18 22:14:20--  https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula7/sample_brwac.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 209.85.147.128, 142.250.125.128, 142.250.136.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|209.85.147.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123983611 (118M) [text/plain]
Saving to: ‘sample_brwac.txt’

sample_brwac.txt    100%[===================>] 118.24M   242MB/s    in 0.5s    

2022-05-18 22:14:21 (242 MB/s) - ‘sample_brwac.txt’ saved [123983611/123983611]



In [9]:
# Load datasets
context_size = 9

valid_examples = 100
test_examples = 100
texts = open('sample_brwac.txt').readlines()

#print('Truncating for debugging purposes.')
#texts = texts[:500]  # trunca dataset para trabalhar só com 500 documentos dele. treinar no dataset inteiro para ter uma boa PPL

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, context_size=context_size)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, context_size=context_size)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, context_size=context_size)

In [10]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 27675945
valid examples: 82070
test examples: 166726


training examples: 406905
valid examples: 135562
test examples: 136690

In [11]:
import torch.nn

class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size, context_size, embedding_dim, hidden_size):
        """
        Implements the Neural Language Model proposed by Bengio et al."

        Args:
            vocab_size (int): Size of the input vocabulary.
            context_size (int): Size of the sequence to consider as context for prediction.
            embedding_dim (int): Dimension of the embedding layer for each word in the context.
            hidden_size (int): Size of the hidden layer.
        """
        # Escreva seu código aqui.
        super(LanguageModel, self).__init__()
        self.vocab_size = vocab_size
        self.context_size = context_size
        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size

        self.embeddings = torch.nn.Embedding(num_embeddings = self.vocab_size, embedding_dim = self.embedding_dim)
        
        # definir camadas totalmente conectadas
        self.linear1 = nn.Linear(context_size * embedding_dim, self.hidden_size * 2)
        self.linear2 = nn.Linear(self.hidden_size * 2, self.hidden_size * 2)
        self.linear3 = nn.Linear(self.hidden_size * 2, vocab_size)

        # testar funções de ativação:
        #self.relu1 = nn.ReLU()
        #self.relu2 = nn.ReLU()

        # tentar essas duas depois
        self.tanh1 = nn.Tanh()
        self.tanh2 = nn.Tanh()

    def forward(self, inputs):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, context_size)
        """
        # Escreva seu código aqui.
        # definir o vetor de entrada na rede
        x = self.embeddings(inputs) #lista de embeddings com os indices dos inputs
        x = x.view(-1,self.context_size*self.embedding_dim) # tornar a lista total anterior como um grande tensor

        x = self.linear1(x)
        x = self.tanh1(x)
        x = self.linear2(x)
        x = self.tanh2(x)
        x = self.linear3(x)
        #out_relu = torch.nn.functional.tanh(x_linear1)
        #logitos = self.linear2(out_relu)
        #self.logitos = logitos
        #log_probs = torch.nn.functional.softmax(logitos, dim=1)
        #self.log_probs = log_probs

        return x


    # verificar alguns resultados
    def print(self):
      print(f'Logitos: {self.logitos}'f'Embeddings: {self.embeddings.weight}', f'Embeddings shape: {self.embeddings.weight.shape}', f'probs: {self.log_probs}')


## Teste o modelo com um exemplo

In [12]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=64,
    hidden_size=128,
).to(device)

sample_train, _ = next(iter(DataLoader(training_dataset)))
sample_train_gpu = sample_train.to(device)
model(sample_train_gpu).shape

#model.print()

torch.Size([1, 29794])

In [13]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 9777378


## Assert da Perplexidade


In [14]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, vocab_size)
        target: a LongTensor of shape (batch_size,)

    Returns:
        A float corresponding to the perplexity.
    """
    # Escreva seu código aqui.
    # getting loss using cross entropy
    loss = torch.nn.functional.cross_entropy(logits, target)

    # calculating perplexity
    perplexity  = torch.exp(loss)
    
    return perplexity

n_examples = 1000

sample_train, target_token_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
sample_train_gpu = sample_train.to(device)
target_token_ids = target_token_ids.to(device)
logits = model(sample_train_gpu)

my_perplexity = perplexity(logits=logits, target=target_token_ids)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=2000)
print('Passou o no assert da perplexidade')

my perplexity:              30133
correct initial perplexity: 29794
Passou o no assert da perplexidade


## Laço de Treinamento e Validação

In [17]:
#max_examples = 100_000_000 #não definimos uma época, sim a quantidade de exemplos que o modelo quer ver

max_examples = 50_000_000 #não definimos uma época, sim a quantidade de exemplos que o modelo quer ver
eval_every_steps = 5000 # validação a cada 5000 passos
lr = 3e-5


model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=128,
    hidden_size=256,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=64, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=64)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input, target):
    model.train()
    model.zero_grad()

    logits = model(input.to(device))
    loss = nn.functional.cross_entropy(logits, target.to(device))
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input, target):
    logits = model(input)
    loss = nn.functional.cross_entropy(logits, target)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for input, target in train_loader:
        loss = train_step(input.to(device), target.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(input.to(device), target.to(device))
                    for input, target in validation_loader]))

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(input)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 29732.91, valid ppl: 30165.04
5000 steps; 320000 examples so far; train ppl: 2652.32, valid ppl: 1698.85
10000 steps; 640000 examples so far; train ppl: 1522.75, valid ppl: 1389.87
15000 steps; 960000 examples so far; train ppl: 1278.85, valid ppl: 1204.70
20000 steps; 1280000 examples so far; train ppl: 1135.07, valid ppl: 1095.04
25000 steps; 1600000 examples so far; train ppl: 1045.50, valid ppl: 1021.43
30000 steps; 1920000 examples so far; train ppl: 984.87, valid ppl: 961.56
35000 steps; 2240000 examples so far; train ppl: 924.91, valid ppl: 914.21
40000 steps; 2560000 examples so far; train ppl: 878.39, valid ppl: 875.56
45000 steps; 2880000 examples so far; train ppl: 841.96, valid ppl: 840.06
50000 steps; 3200000 examples so far; train ppl: 812.53, valid ppl: 806.32
55000 steps; 3520000 examples so far; train ppl: 797.46, valid ppl: 784.27
60000 steps; 3840000 examples so far; train ppl: 758.24, valid ppl: 758.90
65000 steps; 4160000 exam

## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [18]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(input.to(device), target.to(device))
        for input, target in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 233.84482894990916


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [19]:
# usando o modelo para prever sentenças grandes
prompt = 'Eu gosto de comer pizza pois me faz'  # Ex: 'Eu gosto de comer pizza pois me faz'
max_output_tokens = 10

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor(input_ids_truncated).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Eu gosto de comer pizza pois me faz,
Eu gosto de comer pizza pois me faz, mas
Eu gosto de comer pizza pois me faz, mas não
Eu gosto de comer pizza pois me faz, mas não é
Eu gosto de comer pizza pois me faz, mas não é o
Eu gosto de comer pizza pois me faz, mas não é o que
Eu gosto de comer pizza pois me faz, mas não é o que eu
Eu gosto de comer pizza pois me faz, mas não é o que eu não
Eu gosto de comer pizza pois me faz, mas não é o que eu não sei
Eu gosto de comer pizza pois me faz, mas não é o que eu não sei.
